In [ ]:
# -*- coding: utf-8 -*-
import os, re, json, unicodedata, random
from typing import List, Dict, Any, Tuple, Optional
import pandas as pd
import torch

# Make transformers quieter (ostrzeżenia o "temperature" itp.)
try:
    from transformers.utils import logging as hf_logging
    hf_logging.set_verbosity_error()
    os.environ["TRANSFORMERS_VERBOSITY"] = "error"
except Exception:
    pass

# --- LangChain
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_community.retrievers import BM25Retriever

# --- Cross-encoder (reranker) — tylko import flagi
_HAS_RERANK = True
try:
    from langchain_community.cross_encoders import HuggingFaceCrossEncoder
except Exception:
    _HAS_RERANK = False

# --- Chroma (nowe/stare API)
try:
    from langchain_chroma import Chroma
    _CHROMA_NEW = True
except ImportError:
    from langchain_community.vectorstores import Chroma
    _CHROMA_NEW = False

# -----------------------------
# Ścieżki/katalogi
# -----------------------------
CSV_DIR = "all_dane"   # ← podmień na swój folder
PERSIST_DIR = "chroma_statystyki"
RERANKER_MODEL = "BAAI/bge-reranker-v2-m3"
EMBEDDER_MODEL = "intfloat/multilingual-e5-large"
LLM = "CYFRAGOVPL/PLLuM-12B-chat"

# --- MultiQuery ustawienia
LLM_MQ_ENABLED = True
MQ_BASE_VARIANTS = 4
MQ_LLM_VARIANTS = 8

# HyDE – włączony + ustawienia
USE_HYDE = True
HYDE_VARIANTS_BASE = 3
HYDE_VARIANTS_LLM = 3
HYDE_STRIP_NUMBERS = True

# Wagi do fuzji RRF dla poszczególnych biegów
RRF_WEIGHTS = {"dense_q": 1.0, "dense_mq": 0.95, "hyde": 0.7, "bm25": 1.0}

# -----------------------------
# Progi / decyzje
# -----------------------------
MIN_RERANK_SCORE = 0.0          # brak wczesnego progu — decyzja dopiero po CE 
MIN_DOCS_AFTER_FILTER = 1
RRF_LOW_THRESHOLD = 0.0         # wyłączony fallback Top-3
MIN_CE_FOR_CONTEXT = 0.1        # twarde odcięcie: tylko dokumenty z CE

# -----------------------------
# Czy używa modelu do odpowiedzi
# -----------------------------
USE_LLM = True

# -----------------------------
# Debug / śledzenie kontekstu
# -----------------------------
DEBUG_CONTEXT = False
LAST_DEBUG: Dict[str, Any] = {}

def _dbg(msg: str):
    if DEBUG_CONTEXT:
        print(str(msg))

def get_last_debug() -> Dict[str, Any]:
    return LAST_DEBUG

# -----------------------------
# Preferencje wyboru
# -----------------------------
NATIONAL_TOKENS = {
    "ogolem", "ogółem", "OGÓŁEM'," "-", "", "polska", "kraj", "poland",
    "cały kraj", "cala polska", "cała polska", "caly kraj", "cały kraj"
}
FORCE_NATIONAL_IF_REGION_UNSPECIFIED = True  # preferuj „ogółem”, gdy brak regionu w pytaniu
PREFER_TYP_FROM_QUERY = True                 # preferuj typ z intencji (np. „emerytury”)

def strip_acc(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", str(s)) if unicodedata.category(c) != "Mn")

def norm_text(s: str) -> str:
    s = strip_acc(str(s)).lower()
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _is_national(region: Optional[str]) -> bool:
    if region is None:
        return True
    s = norm_text(region)
    s2 = re.sub(r"[^\w ]", "", s)
    return (s2 in NATIONAL_TOKENS) or s2.startswith("ogolem")

def derive_preferences(query: str, parsed: Dict[str, Optional[str]]) -> Dict[str, Any]:
    qn = norm_text(query)
    prefer_typ, force_typ = None, False
    if "emerytaln" in qn or "emerytur" in qn or "emerytura" in qn:
        prefer_typ = "emerytury"
        force_typ = not ("rent" in qn or "renty" in qn or "rentę" in qn or "rentowe" in qn)
    force_national = FORCE_NATIONAL_IF_REGION_UNSPECIFIED and (parsed.get("region") is None)
    return {"prefer_typ": prefer_typ, "force_typ": force_typ, "force_national": force_national}

# --- HyDE helpers (numbers stripping)
NUM_PAT = re.compile(r"(?<![A-Za-z])[-+]?\d[\d\s,./%]*")
def _strip_numbers(text: str) -> str:
    return NUM_PAT.sub("<X>", str(text))

# -----------------------------
# Smart CSV reader (PL encodings)
# -----------------------------
def _read_csv_smart(path: str) -> pd.DataFrame:
    for enc in ["utf-8-sig", "cp1250", "iso-8859-2", "latin-1"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    return pd.read_csv(path)

# -----------------------------
# Parsing liczb i okresów
# -----------------------------
def parse_value(x) -> Optional[float]:
    if x is None: return None
    s = str(x).strip()
    if s == "" or norm_text(s) in {"nan", "brak", "null"}:
        return None
    s = s.replace(" ", "").replace("\u00A0", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None

def norm_period(p: Optional[str]) -> Optional[str]:
    if not p or str(p).strip()=="":
        return None
    s = str(p).replace("\u00A0"," ").strip()
    s = s.upper()
    s = re.sub(r"[R]\.?$", "", s)
    s = re.sub(r"\s+", "", s)
    s = s.replace("/", "-").replace("_","-")
    # 2025-Q1
    if re.match(r"^20\d{2}-Q[1-4]$", s):
        return s
    # 2025Q1
    m = re.match(r"^(20\d{2})Q([1-4])$", s)
    if m:
        return f"{m.group(1)}-Q{m.group(2)}"
    # Q1-2025 / Q1 2025 / Q1_2025
    m = re.match(r"^Q([1-4])[- ]?(20\d{2})$", s)
    if m:
        return f"{m.group(2)}-Q{m.group(1)}"
    return s  # zostaw surowe — będzie mniej premiowane

def clean_measure_and_type(measure: str, typ: Optional[str]) -> Tuple[str, Optional[str]]:
    m = str(measure or "").strip()
    t = None if (typ is None or str(typ).strip()=="") else str(typ).strip()
    # autocorrect
    m = m.replace("przciętna", "przeciętna").replace(" w zl", " w zł")
    # typ w nawiasie → wyciągnij
    paren = re.search(r"\(([^)]+)\)", m)
    if paren and (t is None or t==""):
        t = paren.group(1).strip()
    m = re.sub(r"\s*\([^)]+\)\s*", " ", m)
    m = re.sub(r"\s+", " ", m).strip()
    return m, t

def make_page_text(row: dict) -> str:
    return " | ".join([
        f"dataset: {row.get('dataset')}",
        f"measure: {row.get('measure_clean')}",
        f"value: {row.get('value_float')}",
        f"region: {row.get('region') or '-'}",
        f"period: {row.get('period_norm') or row.get('period_raw') or '-'}",
        f"typ: {row.get('typ_clean') or '-'}",
    ])

# -----------------------------
# Dynamiczny „słownik” uczony z danych
# -----------------------------
FIELD_VOCAB: Dict[str, set[str]] = {"dataset": set(), "measure": set(), "region": set(), "typ": set()}

def _add_to_vocab(val, field):
    if val is None: return
    s = str(val).strip()
    if not s: return
    FIELD_VOCAB[field].add(norm_text(s))

# -----------------------------
# Budowa Document z CSV (schemat standardowy)
# -----------------------------
def build_documents_from_standard_csv(dataset_name: str, csv_path: str) -> List[Document]:
    df = _read_csv_smart(csv_path)
    required = ["dataset","measure","value","region","period","typ"]
    for c in required:
        if c not in df.columns:
            raise ValueError(f"Brak wymaganej kolumny: {c}")
    docs: List[Document] = []
    for i, row in df.iterrows():
        dataset = str(row.get("dataset") or "").strip()
        measure_raw = row.get("measure")
        typ_raw     = row.get("typ")
        measure_clean, typ_clean = clean_measure_and_type(measure_raw, typ_raw)

        value_float = parse_value(row.get("value"))
        region_raw  = None if pd.isna(row.get("region")) else str(row.get("region")).strip()
        period_raw  = None if pd.isna(row.get("period")) else str(row.get("period")).strip()
        period_norm_ = norm_period(period_raw)

        rec = {
            "dataset": dataset or dataset_name,
            "measure_clean": measure_clean,
            "typ_clean": typ_clean,
            "value_float": value_float,
            "region": region_raw,
            "period_raw": period_raw,
            "period_norm": period_norm_,
        }
        page_text = make_page_text(rec)
        meta: Dict[str, Any] = {
            "dataset": dataset or dataset_name,
            "source_file": os.path.basename(csv_path),
            "row_index": int(i),
            "okres": period_norm_ or period_raw,
            "region": region_raw,
            "type": typ_clean,
            "measure": measure_clean,
            "value": value_float,
        }
        docs.append(Document(page_content=page_text, metadata=meta))

        # uczymy słownik
        _add_to_vocab(meta["dataset"], "dataset")
        _add_to_vocab(meta["measure"], "measure")
        _add_to_vocab(meta["region"], "region")
        _add_to_vocab(meta["type"], "typ")
    return docs

# -----------------------------
# EMBEDDINGS — E5
# -----------------------------
from transformers import AutoModel, AutoTokenizer

class MXBAIEmbeddings(Embeddings):
    def __init__(self, model_id: str = EMBEDDER_MODEL, device: Optional[str] = None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.tok = AutoTokenizer.from_pretrained(model_id)
        self.net = AutoModel.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16 if self.device=="cuda" else torch.float32
        ).to(self.device)
        self.net.eval()

    @torch.inference_mode()
    def _encode(self, texts: List[str], is_query=False, batch_size=32) -> List[List[float]]:
        prefix = "query: " if is_query else "passage: "
        out = []
        for i in range(0, len(texts), batch_size):
            batch = [prefix + t for t in texts[i:i+batch_size]]
            enc = self.tok(batch, padding=True, truncation=True, return_tensors="pt").to(self.device)
            last = self.net(**enc).last_hidden_state
            attn = enc["attention_mask"].unsqueeze(-1)
            emb = (last * attn).sum(dim=1) / torch.clamp(attn.sum(dim=1), min=1)
            emb = torch.nn.functional.normalize(emb, p=2, dim=1)
            out.extend(emb.detach().cpu().tolist())
        return out

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self._encode(texts, is_query=False)

    def embed_query(self, text: str) -> List[float]:
        return self._encode([text], is_query=True)[0]

# -----------------------------
# Załaduj wszystkie CSV
# -----------------------------
def _looks_standard(df: pd.DataFrame) -> bool:
    needed = {"dataset","measure","value","region","period","typ"}
    return needed.issubset(set(df.columns))

CSV_SOURCES = {}
if os.path.isdir(CSV_DIR):
    for fn in os.listdir(CSV_DIR):
        if fn.lower().endswith(".csv"):
            CSV_SOURCES[os.path.splitext(fn)[0]] = os.path.join(CSV_DIR, fn)
elif os.path.isfile(CSV_DIR) and CSV_DIR.lower().endswith(".csv"):
    CSV_SOURCES[os.path.splitext(os.path.basename(CSV_DIR))[0]] = CSV_DIR
else:
    print(f"[UWAGA] {CSV_DIR} nie jest ani katalogiem z CSV, ani plikiem CSV.")

all_docs: List[Document] = []
for name, path in CSV_SOURCES.items():
    try:
        df_head = _read_csv_smart(path).head(1)
        if _looks_standard(df_head):
            docs = build_documents_from_standard_csv(name, path)
            all_docs.extend(docs)
        else:
            print(f"[UWAGA] Pomijam '{path}' — niestandardowy schemat kolumn.")
    except Exception as e:
        print(f"[BŁĄD] {path}: {e}")

print(f"Zbudowano dokumentów: {len(all_docs)}")

# -----------------------------
# Wektoryzacja Chroma
# -----------------------------
emb = MXBAIEmbeddings()
vectorstore = Chroma(
    collection_name="statystyki",
    embedding_function=emb,
    persist_directory=PERSIST_DIR
)

def _collection_count(vs) -> int:
    coll = getattr(vs, "_collection", None)
    try: return coll.count() if coll is not None else 0
    except Exception: return 0

if _collection_count(vectorstore) == 0 and all_docs:
    vectorstore.add_documents(all_docs)
    getattr(vectorstore, "persist", lambda: None)()

# -----------------------------
# BM25 retriever
# -----------------------------
bm25 = BM25Retriever.from_documents(all_docs)
bm25.k = 80

# -----------------------------
# MultiQuery + HyDE
# -----------------------------
def _base_mq_variants(q: str, n: int = MQ_BASE_VARIANTS) -> List[str]:
    qn = norm_text(q)
    base = {
        q.strip(),
        strip_acc(q),
        qn.replace("ile", "jaka jest liczba"),
        qn.replace("ile wynosi", "podaj wartość"),
    }
    # dopalacze MQ dla typowych intencji
    if "ubezpiecz" in qn:
        base.update({
            "liczba ubezpieczonych ogółem",
            "liczba ubezpieczonych w krus ogółem",
            "ubezpieczeni w krus ogółem"
        })
    if "płatnik" in qn or "platnik" in qn:
        base.update({"liczba płatników składek ogółem"})
    if "macierzyńsk" in qn or "macierzynsk" in qn:
        base.update({"przeciętne świadczenie zasiłek macierzyński ogółem"})
    # hint „ogółem”, gdy brak regionu
    if FORCE_NATIONAL_IF_REGION_UNSPECIFIED and "wojew" not in qn and "region" not in qn:
        base.add(q.strip() + " ogółem")
    return list(base)[:n]

def _take_closest_vocab(target: Optional[str], pool: set[str], k: int = 20) -> List[str]:
    if not target or not pool: return []
    scored = []
    for cand in pool:
        scored.append((cand, _ratio(target, cand)))
    scored.sort(key=lambda x: x[1], reverse=True)
    return [c for c, _ in scored[:k]]

def _safe_unique(xs: List[str], limit: int = 20) -> List[str]:
    seen, out = set(), []
    for x in xs:
        s = norm_text(x)
        if s and s not in seen:
            seen.add(s); out.append(x.strip())
        if len(out) >= limit: break
    return out

def _pick_or_default(val: Optional[str], pool: set[str], k: int = 3) -> List[str]:
    base = [val] if val else []
    rest = _take_closest_vocab(val, pool, k=k) if pool else []
    out = [x for x in base + rest if x]
    return _safe_unique(out, limit=max(1, k+1)) or ["-"]

def _llm_generate_query_expansions(question: str,
                                   measure_hint: Optional[str],
                                   typ_hint: Optional[str],
                                   dataset_hint: Optional[str],
                                   k_variants: int = MQ_LLM_VARIANTS) -> List[str]:
    if not (USE_LLM and llm_pllum and LLM_MQ_ENABLED):
        return []

    measure_cands = _take_closest_vocab(measure_hint, FIELD_VOCAB["measure"], k=20)
    typ_cands     = _take_closest_vocab(typ_hint,     FIELD_VOCAB["typ"],     k=20)
    dataset_cands = _take_closest_vocab(dataset_hint, FIELD_VOCAB["dataset"], k=20)

    sys_rules = (
        "Jesteś generatorem wariantów zapytań do wyszukiwarki danych KRUS.\n"
        "Zasady:\n"
        "• Zwięzłe warianty PL (jedna linia = jedno zapytanie).\n"
        "• Używaj TYLKO podanych etykiet measure/typ/dataset (jeśli podane).\n"
        "• Zachowuj znaczenie pytania.\n"
        "• Nie zgaduj, gdy brak kandydatów.\n"
    )
    m_line = " | ".join(measure_cands) if measure_cands else "(brak)"
    t_line = " | ".join(typ_cands) if typ_cands else "(brak)"
    d_line = " | ".join(dataset_cands) if dataset_cands else "(brak)"

    user_block = (
        f"Pytanie:\n{question}\n\n"
        f"Kandydaci 'measure': {m_line}\n"
        f"Kandydaci 'typ': {t_line}\n\n"
        f"Kandydaci 'dataset': {d_line}\n"
        f"Wygeneruj do {k_variants} wariantów. Zwróć TYLKO linie zapytań."
    )
    prompt = f"<s>[INST] <<SYS>>{sys_rules}<</SYS>>\n{user_block}\n[/INST]"
    try:
        out = llm_pllum.invoke(prompt)
        lines = [ln.strip(" -•\t") for ln in str(out).splitlines()]
        lines = [ln for ln in lines if ln and not ln.lower().startswith(("[źródła", "zrodla", "sources", "<", "["))]
        return _safe_unique(lines, limit=k_variants)
    except Exception:
        return []

def make_mq_prompts_llm(q: str) -> List[str]:
    parsed = parse_query_fields(q)
    base = _safe_unique(_base_mq_variants(q, n=MQ_BASE_VARIANTS))
    gen  = _llm_generate_query_expansions(
        question=q,
        measure_hint=parsed.get("measure"),
        typ_hint=parsed.get("typ"),
        dataset_hint=parsed.get("dataset"),
        k_variants=MQ_LLM_VARIANTS
    )
    return _safe_unique(base + gen, limit=MQ_BASE_VARIANTS + MQ_LLM_VARIANTS)

def make_hyde_texts(question: str,
                    n_base: int = HYDE_VARIANTS_BASE,
                    n_llm:  int = HYDE_VARIANTS_LLM) -> List[str]:
    parsed = parse_query_fields(question)
    m  = parsed.get("measure")
    t  = parsed.get("typ")
    ds = parsed.get("dataset")
    rg = parsed.get("region") or ("ogółem" if FORCE_NATIONAL_IF_REGION_UNSPECIFIED else "-")
    pr = parsed.get("period") or "-"

    cand_meas = _pick_or_default(m,  FIELD_VOCAB["measure"], k=3)
    cand_typ  = _pick_or_default(t,  FIELD_VOCAB["typ"],     k=3)
    cand_ds   = _pick_or_default(ds, FIELD_VOCAB["dataset"], k=3)

    base: List[str] = []
    for i in range(max(1, n_base)):
        base.append(" | ".join([
            f"dataset: {cand_ds[i % len(cand_ds)]}",
            f"measure: {cand_meas[i % len(cand_meas)]}",
            "value: <X>",
            f"region: {rg}",
            f"period: {pr}",
            f"typ: {cand_typ[i % len(cand_typ)]}",
        ]))

    gen: List[str] = []
    if USE_LLM and llm_pllum and n_llm > 0:
        sys = (
            "Jesteś pomocnikiem generującym hipotetyczne opisy danych KRUS. "
            "Napisz 1–2 zdania po polsku, bez żadnych liczb/%, bez konkretnych wartości. "
            "Jeśli pojawia się liczba, zastąp ją tokenem <X>."
        )
        user = (
            f"Pytanie: {question}\n"
            f"Wskazówki: dataset={ds or '-'}, measure={m or '-'}, typ={t or '-'}, region={rg}, period={pr}.\n"
            f"Wypisz {n_llm} wariantów, każdy w osobnej linii."
        )
        prompt = f"<s>[INST] <<SYS>>{sys}<</SYS>>\n{user}\n[/INST]"
        try:
            raw = str(llm_pllum.invoke(prompt))
            lines = [ln.strip(" -•\t") for ln in raw.splitlines() if ln.strip()]
            gen = lines[:n_llm]
        except Exception:
            gen = []

    texts = base + gen
    if HYDE_STRIP_NUMBERS:
        texts = [_strip_numbers(t) for t in texts]
    return _safe_unique(texts, limit=n_base + n_llm)

# -----------------------------
# RRF — bazowy + ważony
# -----------------------------
def rrf_merge_with_support(runs: List[List[Document]], k: int = 120, k_rrf: int = 60) -> List[Document]:
    scores, support, best = {}, {}, {}
    pos_maps: List[Dict[Tuple[str,int], int]] = []
    for run in runs:
        m = {}
        for i, d in enumerate(run):
            key = (d.metadata.get("source_file","?"), d.metadata.get("row_index",-1))
            m[key] = i
        pos_maps.append(m)
    keys = set().union(*[set(m.keys()) for m in pos_maps])
    for key in keys:
        rrf = 0.0; sup = 0
        for m in pos_maps:
            if key in m:
                rrf += 1.0 / (k_rrf + m[key] + 1)
                sup += 1
        scores[key]  = rrf
        support[key] = sup
    for run in runs:
        for d in run:
            key = (d.metadata.get("source_file","?"), d.metadata.get("row_index",-1))
            if key not in best: best[key] = d
    merged = sorted(best.values(),
                    key=lambda d: scores[(d.metadata.get("source_file","?"), d.metadata.get("row_index",-1))],
                    reverse=True)
    for d in merged:
        key = (d.metadata.get("source_file","?"), d.metadata.get("row_index",-1))
        d.metadata["rrf_support"] = support[key]
        d.metadata["rrf_score"]   = scores[key]
    return merged[:k]

def rrf_merge_with_support_weighted(runs: List[List[Document]],
                                    labels: List[str],
                                    weights: Dict[str, float],
                                    k: int = 120,
                                    k_rrf: int = 60) -> List[Document]:
    scores, support_w, best = {}, {}, {}
    pos_maps: List[Dict[Tuple[str,int], int]] = []

    for run in runs:
        m = {}
        for i, d in enumerate(run):
            key = (d.metadata.get("source_file","?"), d.metadata.get("row_index",-1))
            m[key] = i
        pos_maps.append(m)

    keys = set().union(*[set(m.keys()) for m in pos_maps])
    for key in keys:
        s = 0.0; supw = 0.0
        for m, lab in zip(pos_maps, labels):
            if key in m:
                w = float(weights.get(lab, 1.0))
                s   += w * (1.0 / (k_rrf + m[key] + 1))
                supw += w
        scores[key]    = s
        support_w[key] = supw

    for run in runs:
        for d in run:
            key = (d.metadata.get("source_file","?"), d.metadata.get("row_index",-1))
            if key not in best:
                best[key] = d

    merged = sorted(best.values(),
                    key=lambda d: scores[(d.metadata.get("source_file","?"), d.metadata.get("row_index",-1))],
                    reverse=True)
    for d in merged:
        key = (d.metadata.get("source_file","?"), d.metadata.get("row_index",-1))
        d.metadata["rrf_score"]   = scores[key]
        d.metadata["rrf_support"] = support_w[key]  # „ważone” wsparcie
    return merged[:k]

# -----------------------------
# Dense search helpers
# -----------------------------
def dense_search(query: str, k: int = 80) -> List[Document]:
    retr = vectorstore.as_retriever(search_kwargs={"k": k})
    return retr.invoke(query)

def dense_search_on(texts: List[str], k: int = 40) -> List[Document]:
    retr = vectorstore.as_retriever(search_kwargs={"k": k})
    out = []
    for t in texts:
        out.extend(retr.invoke(t))
    seen = set()
    uniq = []
    for d in out:
        key = (d.metadata.get("source_file","?"), d.metadata.get("row_index",-1))
        if key in seen: continue
        seen.add(key); uniq.append(d)
    return uniq

# -----------------------------
# Fuzzy dopasowanie pól
# -----------------------------
try:
    from rapidfuzz import fuzz
    def _ratio(a,b): return fuzz.token_set_ratio(a,b) / 100.0
except Exception:
    import difflib
    def _ratio(a,b): return difflib.SequenceMatcher(None, a, b).ratio()

def best_match(token: str, candidates: set[str], min_ratio: float = 0.62) -> Optional[str]:
    if not token or not candidates: return None
    tn = norm_text(token)
    best, score = None, 0.0
    for c in candidates:
        r = _ratio(tn, c)
        if r > score:
            best, score = c, r
    return best if score >= min_ratio else None

def parse_query_fields(q: str) -> Dict[str, Optional[str]]:
    qn = norm_text(q)
    m = re.search(r"(20\d{2}\s*[-_/ ]?\s*q[1-4])", qn)
    period = norm_period(m.group(1)) if m else None
    dataset = best_match(qn, FIELD_VOCAB["dataset"])
    measure = best_match(qn, FIELD_VOCAB["measure"])
    region  = best_match(qn, FIELD_VOCAB["region"])
    typ     = best_match(qn, FIELD_VOCAB["typ"])
    if typ and measure and norm_text(typ) == norm_text(measure):
        typ = None
    return {"dataset": dataset, "measure": measure, "region": region, "typ": typ, "period": period}

# -----------------------------
# Reranker CE (z punktacją)  — instancja po emb!
# -----------------------------
from sentence_transformers import CrossEncoder

CE_MODEL_ID = RERANKER_MODEL
reranker_ce = None
try:
    reranker_ce = CrossEncoder(CE_MODEL_ID, device=emb.device)
except Exception:
    reranker_ce = None

def _ce_text(d: Document, n: int = 800) -> str:
    s = d.page_content or ""
    return s if len(s) <= n else s[:n]

def _to_float_score(sc) -> float:
    try:
        import numpy as np
        if hasattr(sc, "shape"):
            return float(np.squeeze(sc))
    except Exception:
        pass
    if isinstance(sc, (list, tuple)):
        return 0.0 if not sc else _to_float_score(sc[0])
    try:
        return float(sc)
    except Exception:
        return 0.0

def _fallback_dense_similarity(query: str, docs: List[Document]) -> List[Tuple[Document,float]]:
    qv = torch.tensor(emb.embed_query(query))
    dvs = torch.tensor([emb.embed_documents([d.page_content])[0] for d in docs])
    sims = torch.nn.functional.cosine_similarity(qv[None, :], dvs, dim=1).tolist()
    ranked = sorted([(d, s) for d, s in zip(docs, sims)], key=lambda x: x[1], reverse=True)
    for d, s in ranked:
        try: d.metadata["ce_score"] = float(s)
        except Exception: pass
    return ranked

def rerank_with_scores(query: str, docs: List[Document]) -> List[Tuple[Document, float]]:
    if reranker_ce is None or not docs:
        for d in docs:
            try: d.metadata["ce_score"] = 0.0
            except Exception: pass
        return [(d, 0.0) for d in docs]

    pairs = [(query, _ce_text(d)) for d in docs]
    scores = reranker_ce.predict(pairs)
    out = []
    for d, sc in zip(docs, scores):
        scf = _to_float_score(sc)
        try: d.metadata["ce_score"] = scf
        except Exception: pass
        out.append((d, scf))

    ranked = sorted(out, key=lambda x: x[1], reverse=True)

    vals = [s for _, s in ranked]
    if len(vals) >= 3 and (max(vals) - min(vals) < 1e-6):
        ranked = _fallback_dense_similarity(query, docs)

    return ranked

# -----------------------------
# Okres → sortowanie
# -----------------------------
def period_key(p: Optional[str]) -> Tuple[int,int,int]:
    if not p or str(p).strip()=="": return (0,0,0)
    s = str(p).lower().strip()
    m = re.match(r"^(20\d{2})[-_/ ]?q([1-4])$", s)
    if m: return (int(m.group(1)), int(m.group(2)), 0)
    m = re.match(r"^q([1-4])[-_/ ]?(20\d{2})$", s)
    if m: return (int(m.group(2)), int(m.group(1)), 0)
    m = re.match(r"^(20\d{2})[-_/](\d{1,2})[-_/](\d{1,2})$", s)
    if m:
        y, mo, d = map(int, m.groups())
        daynum = (mo-1)*31 + d
        q = (mo-1)//3 + 1
        return (y, q, daynum)
    m = re.match(r"^(20\d{2})$", s)
    if m: return (int(m.group(1)), 0, 0)
    return (0,0,0)

# -----------------------------
# Klastrowanie i wybór „najnowszego ≠ NaN”
# -----------------------------
def _key4(d: Document) -> Tuple[str,str,str,str]:
    m = d.metadata or {}
    return (
        norm_text(m.get("dataset") or ""),
        norm_text(m.get("measure") or ""),
        norm_text(m.get("type") or ""),
        norm_text(m.get("region") or ""),
    )

def _valid_value(d: Document) -> bool:
    v = d.metadata.get("value", None)
    try:
        return (v is not None) and (not (isinstance(v,float) and (pd.isna(v))))
    except Exception:
        return v is not None

def pick_latest_per_cluster(
    docs_scored: List[Tuple[Document, float]],
    k_clusters: int = 6,
    prefer_typ: Optional[str] = None,
    prefer_national_first: bool = False
) -> List[Document]:
    bykey: Dict[Tuple[str,str,str,str], List[Tuple[Document,float,int]]] = {}
    for d, s in docs_scored:
        sup = int(d.metadata.get("rrf_support", 1))
        bykey.setdefault(_key4(d), []).append((d, s, sup))

    best_per: List[Tuple[Document,float,int]] = []
    for key, items in bykey.items():
        # sort lokalny: wsparcie RRF, score (tu CE), okres
        items.sort(key=lambda x: (x[2], x[1], period_key(x[0].metadata.get("okres"))), reverse=True)
        items_valid = [it for it in items if _valid_value(it[0])]
        chosen = (sorted(items_valid, key=lambda x: period_key(x[0].metadata.get("okres")), reverse=True)[0]
                  if items_valid else items[0])
        best_per.append(chosen)

    def _prio(meta: Dict[str, Any]) -> Tuple[int, int]:
        region_prio = 1 if (prefer_national_first and _is_national(meta.get("region"))) else 0
        typ_prio = 1 if (prefer_typ and _soft_match(prefer_typ, meta.get("type"))) else 0
        return (region_prio, typ_prio)

    # globalnie: ogółem/typ preferowany, potem wsparcie/CE/świeżość
    best_per.sort(
        key=lambda x: (
            _prio(x[0].metadata),
            x[2],                                  # rrf_support
            x[1],                                  # score = CE
            period_key(x[0].metadata.get("okres")) # świeżość
        ),
        reverse=True
    )
    return [d for d,_,_ in best_per[:k_clusters]]

# -----------------------------
# Penalizacje datasetów + utilsy
# -----------------------------
DATASET_PENALTIES: List[Tuple[re.Pattern, float]] = [
    (re.compile(r"jednoczesnym|jednoczesne|zus", re.I), 0.35),
    (re.compile(r"fundusz\s+składkowy", re.I), 0.30),
    (re.compile(r"transferowan", re.I), 0.40),
    (re.compile(r"ue|efta|wielk(?:a|iej)\s+bryt", re.I), 0.40),
]

# Gating dla zbiorów „zagranicznych” – odrzucaj, jeśli pytanie o nie nie prosi
_FOREIGN_PAT = re.compile(r"\b(ue|efta|wielk(?:a|iej)\s*bryt|dwustronn|transferowan)\b", re.I)
def _dataset_is_foreign(name: Optional[str]) -> bool:
    return bool(_FOREIGN_PAT.search(str(name or "")))
def _query_mentions_foreign(q: str) -> bool:
    return bool(_FOREIGN_PAT.search(norm_text(q)))

def _soft_match(a: Optional[str], b: Optional[str], thr: float = 0.72) -> bool:
    if not a:
        return True
    if not b:
        return False
    return _ratio(norm_text(a), norm_text(b)) >= thr

# --- Filtr regionu
def _region_filter_ok(parsed_region: Optional[str],
                      doc_region: Optional[str],
                      force_national: bool = False) -> bool:
    if force_national and (parsed_region is None or str(parsed_region).strip() == ""):
        return _is_national(doc_region)
    return _soft_match(parsed_region, doc_region)

def _adjusted_rank_score(d: Document, base_ce: float, prefs: Dict[str, Any], parsed: Dict[str, Optional[str]], query: str) -> float:
    score = float(base_ce or 0.0)
    name  = str((d.metadata or {}).get("dataset") or "")
    # penalizacje datasetów „pobocznych”
    for pat, pen in DATASET_PENALTIES:
        if pat.search(name):
            score *= (1.0 - pen)
    # preferuj ogółem gdy brak regionu w pytaniu
    if prefs.get("force_national") and not _is_national((d.metadata or {}).get("region")):
        score *= 0.62
    # lekkie preferowanie zbiorów „w KRUS”, gdy pytanie zawiera „w krus”
    qn = norm_text(query)
    if "krus" in qn and (" w krus" in qn or "wkrus" in qn):
        if re.search(r"\bw\s+krus\b", norm_text(name)):
            score *= 1.1
    return score

def _choose_best_doc(docs: List[Document], query: str, parsed: Dict[str, Optional[str]], prefs: Dict[str, Any]) -> Optional[Document]:
    if not docs:
        return None
    docs_fresh = sorted(docs, key=lambda d: period_key((d.metadata or {}).get("okres")), reverse=True)
    scored: List[Tuple[Document, float]] = []
    for d in docs_fresh:
        ce = float((d.metadata or {}).get("ce_score") or 0.0)
        adj = _adjusted_rank_score(d, ce, prefs, parsed, query)
        scored.append((d, adj))
    scored.sort(key=lambda x: (
        1.0 if _soft_match(parsed.get("measure"), (x[0].metadata or {}).get("measure")) else 0.0,
        1.0 if _soft_match(parsed.get("typ"),     (x[0].metadata or {}).get("type"))    else 0.0,
        1.0 if (prefs.get("force_national") and _is_national((x[0].metadata or {}).get("region"))) else 0.0,
        x[1],  # ce_adj
        period_key((x[0].metadata or {}).get("okres"))
    ), reverse=True)
    return scored[0][0] if scored else None

# -----------------------------
# SMART RETRIEVE (szeroko → sort CE → odcięcie CE < 0.6)
# -----------------------------
def retrieve(query: str, k_final: int = 24) -> List[Document]:
    global LAST_DEBUG

    parsed = parse_query_fields(query)
    prefs = derive_preferences(query, parsed)

    debug_info: Dict[str, Any] = {"query": query, "parsed": parsed, "prefs": prefs}

    # --- MultiQuery / HyDE / BM25 / Dense
    mq_variants = make_mq_prompts_llm(query)
    d1 = dense_search_on(mq_variants, k=60)

    d2 = []
    if USE_HYDE:
        hyde_texts = make_hyde_texts(query, n_base=HYDE_VARIANTS_BASE, n_llm=HYDE_VARIANTS_LLM)
        d2 = dense_search_on(hyde_texts, k=60)

    d0 = dense_search(query, k=100)
    bm25.k = 100
    d3 = bm25.invoke(query)

    debug_info["hits"] = {"dense_q": len(d0), "dense_mq": len(d1), "hyde": len(d2), "bm25": len(d3)}

    fused = rrf_merge_with_support_weighted(
        [d0, d1, d2, d3],
        labels=["dense_q", "dense_mq", "hyde", "bm25"],
        weights=RRF_WEIGHTS,
        k=160,
        k_rrf=60
    )
    docs_scored = rerank_with_scores(query, fused)  # [(doc, ce_score)]

    # Twarde wymuszenie typu z intencji (opcjonalnie)
    if PREFER_TYP_FROM_QUERY and prefs.get("force_typ") and prefs.get("prefer_typ"):
        parsed["typ"] = prefs["prefer_typ"]

    # REGUŁA: szeroko → bierzemy cały ranking CE (bez wczesnych filtrów po polach)
    cand_for_threshold = docs_scored

    # Preferencja „ogółem”, ale nie czyścimy listy, jeśli brak takich rekordów
    if prefs.get("force_national"):
        nat_only = [(d, s) for (d, s) in cand_for_threshold if _is_national((d.metadata or {}).get("region"))]
        if nat_only:
            cand_for_threshold = nat_only

    # Wytnij zbiory „zagraniczne”, jeśli pytanie o nie nie prosi
    if not _query_mentions_foreign(query):
        cand_for_threshold = [
            (d, s) for (d, s) in cand_for_threshold
            if not _dataset_is_foreign((d.metadata or {}).get("dataset"))
        ]

    # Brak wczesnego progu „answerability” — dopiero twarde CE ≥ 0.6
    cand_pairs: List[Tuple[Document, float]] = []
    for d, s in cand_for_threshold:
        ce = float(s or 0.0)
        d.metadata["ce_score"] = ce
        d.metadata["ce_adj"]   = _adjusted_rank_score(d, ce, prefs, parsed, query)  # tylko dla debug/analizy
        cand_pairs.append((d, ce))
    cand_pairs.sort(key=lambda x: x[1], reverse=True)

    # Twarde odcięcie po CE: wyrzuć dokumenty z CE < 0.6
    cand_pairs = [(d, ce) for (d, ce) in cand_pairs if ce >= MIN_CE_FOR_CONTEXT]
    if not cand_pairs:
        debug_info["decision"] = "unanswerable_by_ce_ctx_threshold"
        LAST_DEBUG = debug_info
        return []

    # Klastrowanie po (dataset, measure, typ, region) i wybór najnowszych
    docs_for_llm = pick_latest_per_cluster(
        cand_pairs,
        k_clusters=max(1, k_final),
        prefer_typ=prefs.get("prefer_typ"),
        prefer_national_first=prefs.get("force_national", False)
    )

    # Finalny porządek — najnowszy okres
    docs_for_llm.sort(key=lambda d: period_key(d.metadata.get("okres")), reverse=True)

    # Debug kontekstu
    try:
        selected_headers = []
        for i, d in enumerate(docs_for_llm, 1):
            m = d.metadata or {}
            selected_headers.append({
                "i": i,
                "dataset": m.get("dataset"),
                "source": m.get("source_file"),
                "okres": m.get("okres"),
                "region": m.get("region"),
                "row": m.get("row_index"),
                "ce": m.get("ce_score"),
                "ce_adj": m.get("ce_adj"),
                "rrf_support": m.get("rrf_support"),
                "rrf": m.get("rrf_score"),
            })
        ctx_for_llm = format_context_for_llm(docs_for_llm, max_docs=min(8, len(docs_for_llm)))
        ctx_debug   = format_context_for_debug(docs_for_llm, max_docs=min(8, len(docs_for_llm)))

        debug_info["decision"] = "answerable"
        debug_info["selected_headers"] = selected_headers
        debug_info["ctx_for_llm"] = ctx_for_llm
        debug_info["ctx_debug"] = ctx_debug
        LAST_DEBUG = debug_info
    except Exception as e:
        _dbg(f"[DEBUG] Nie udało się zbudować pakietu debug: {e}")

    # Zwróć tyle, ile się da (do k_final) — wszystkie mają CE ≥ 0.6
    return docs_for_llm[:k_final]

# -----------------------------
# LLM (PLLuM-12B-chat) — do parafraz
# -----------------------------
from transformers import AutoTokenizer as HF_AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_huggingface import HuggingFacePipeline

def build_pllum_llm(model_id: str = LLM, use_4bit: bool = True, max_new_tokens: int = 220):
    if use_4bit:
        bnb = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        llm_model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map="auto", quantization_config=bnb, trust_remote_code=True
        )
    else:
        llm_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True)

    tok = HF_AutoTokenizer.from_pretrained(model_id, use_fast=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

    gen_pipe = pipeline(
        "text-generation",
        model=llm_model,
        tokenizer=tok,
        max_new_tokens=max_new_tokens,
        do_sample=False,  # deterministycznie
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
        return_full_text=False,
    )
    return HuggingFacePipeline(pipeline=gen_pipe)

llm_pllum = build_pllum_llm() if USE_LLM else None

def _trim(s: str, n: int = 500) -> str:
    s = str(s)
    return s if len(s) <= n else s[:n].rstrip() + "…"

def _fmt_num(x, nd=3):
    try:
        return f"{float(x):.{nd}f}"
    except Exception:
        return str(x) if x is not None else "-"

def format_context_for_llm(docs: List[Document], max_docs: int = 6, max_snip_chars: int = 220) -> str:
    lines = []
    for i, d in enumerate(docs[:max_docs], 1):
        meta = d.metadata or {}
        header = (
            f"[{i}] dataset={meta.get('dataset','')} | okres={meta.get('okres') or '-'} | region={meta.get('region') or '-'} "
            f"| row={meta.get('row_index','-')} | type={meta.get('type') or '-'} | measure={meta.get('measure','')} | value={meta.get('value','-')}"
        )
        body = _trim(d.page_content, max_snip_chars)
        lines.append(header + "\n" + body)
    return "\n\n".join(lines)

def format_context_for_debug(docs: List[Document], max_docs: int = 8, max_snip_chars: int = 300) -> str:
    lines = []
    for i, d in enumerate(docs[:max_docs], 1):
        meta = d.metadata or {}
        header = (
            f"[{i}] dataset={meta.get('dataset','')} | source={meta.get('source_file','')} "
            f"| okres={meta.get('okres') or '-'} | region={meta.get('region') or '-'} | row={meta.get('row_index','-')} | type={meta.get('type') or '-'} | measure={meta.get('measure','')} | value={meta.get('value','-')} "
            f"| ce={_fmt_num(meta.get('ce_score'))} | ce_adj={_fmt_num(meta.get('ce_adj'))} | rrf_sup={meta.get('rrf_support','-')} | rrf={_fmt_num(meta.get('rrf_score'),4)}"
        )
        body = _trim(d.page_content, max_snip_chars)
        lines.append(header + "\n" + body)
    return "\n\n".join(lines)

# -----------------------------
# Deterministyczny format odpowiedzi
# -----------------------------
def _pl_number(x: float) -> str:
    if x is None: return "-"
    if abs(x - int(x)) < 1e-6:
        s = f"{int(round(x)):,}".replace(",", " ")
    else:
        s = f"{x:,.2f}".replace(",", " ").replace(".", ",")
    return s

def _unit_from_measure(measure: str) -> str:
    m = norm_text(measure or "")
    if " zł" in (measure or "") or "zł" in m:
        return " zł"
    if "osób" in m or "liczba" in m or "osoby" in m:
        return ""
    return ""

def build_sources_block(docs: List[Document]) -> str:
    lines = ["[ŹRÓDŁA]"]
    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        lines.append(
            f"- #[{i}] dataset={m.get('dataset')} | measure={m.get('measure')}|"
            f"okres={m.get('okres') or '-'} | region={m.get('region') or '-'}  | typ={m.get('type') or '-'} | value={m.get('value')}| "
            f"ce={_fmt_num(m.get('ce_score'))} | ce_adj={_fmt_num(m.get('ce_adj'))} | rrf_sup={m.get('rrf_support','-')} | rrf={_fmt_num(m.get('rrf_score'),4)} | "
            f"source={m.get('source_file')} | row={m.get('row_index','-')}"
        )
    return "\n".join(lines)

# -----------------------------
# Odpowiadanie 
# -----------------------------
def answer(question: str, k_ctx: int = 8) -> str:
    docs = retrieve(question, k_final=max(12, k_ctx))
    if not docs:
        return "Brak danych pasujących do pytania.\n[ŹRÓDŁA]\n– brak"

    # Parsowanie + preferencje do decyzji
    parsed = parse_query_fields(question)
    prefs  = derive_preferences(question, parsed)

    docs_sorted = sorted(docs, key=lambda d: period_key(d.metadata.get("okres")), reverse=True)

    best = _choose_best_doc(docs_sorted, question, parsed, prefs)
    if not best:
        return "Brak danych w dostarczonej dokumentacji.\n[ŹRÓDŁA]\n– brak"

    mv = best.metadata or {}
    val = mv.get("value")
    measure = mv.get("measure") or ""
    okres = mv.get("okres") or "-"
    region = mv.get("region") or "ogółem"
    unit = _unit_from_measure(measure)
    value_text = _pl_number(val) + unit

    answer = f"{value_text} ({okres}, {region})."
    # Dołącz źródła (pierwsze: wybrany; dalej: kilka kontekstowych)
    src_block = build_sources_block([best] + [d for d in docs_sorted if d is not best][:min(5, len(docs_sorted)-1)])
    return answer + "\n" + src_block


Zbudowano dokumentów: 2805


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:

tests = [
"ile wynosi przeciętna liczba świadczeń emerytalnych?",
"ile wynosi przeciętne świadczenie emerytalne?",
"ilu jest płatników składek do krus?",
"ile osób jest ubezpieczone w krus?",
"ile współmałżonków jest ubezpieczone w krus?",
"ile wynosi przeciętna liczba Renty z tytułu niezdolności do pracy razem?",
"ile wypłacono na renty socjalne w tym roku?",
"ile osob podlega ubezpieczeniu zdrowotnemu w kujawsko-pomorskie?",
"ile wynosi przeciętny zasiłek macierzyński?",
]
for q in tests:
    print("\nQ:", q)
    print(answer(q, k_ctx=10))


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Q: ile wynosi przeciętna liczba świadczeń emerytalnych?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Odpowiedź: 963679 (2025-Q1, ogółem).
[ŹRÓDŁA]
- #[1] dataset=EMERYTURY I RENTY REALIZOWANE PRZEZ KRUS | measure=przeciętna miesięczna liczba świadczeniobiorców emerytur i rent|okres=2025-Q1 | region=ogółem  | typ=emerytury | value=760307.0| ce=0.796 | rrf_sup=1.0 | rrf=0.0133 | source=all_data.csv | row=1064
- #[2] dataset=EMERYTURY I RENTY REALIZOWANE PRZEZ KRUS | measure=przeciętna miesięczna liczba świadczeniobiorców|okres=2025-Q1 | region=ogółem  | typ=emerytury | value=None| ce=0.116 | rrf_sup=1.0 | rrf=0.0152 | source=all_data.csv | row=1079
- #[3] dataset=EMERYTURY I RENTY REALIZOWANE PRZEZ KRUS | measure=przeciętna miesięczna liczba świadczeniobiorców emerytur i rent|okres=2025-Q1 | region=ogółem  | typ=ogółem | value=963679.0| ce=0.822 | rrf_sup=1.0 | rrf=0.0141 | source=all_data.csv | row=1061
- #[4] dataset=EMERYTURY I RENTY REALIZOWANE PRZEZ KRUS | measure=przeciętna miesięczna liczba świadczeniobiorców emerytur i rent|okres=2025-Q1 | region=ogółem  | typ=renty rodzinne | v

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Brak danych pasujących do pytania.
[ŹRÓDŁA]
– brak

Q: ile osob podlega ubezpieczeniu zdrowotnemu w kujawsko-pomorskie?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Brak danych pasujących do pytania.
[ŹRÓDŁA]
– brak

Q: ile wynosi przeciętny zasiłek macierzyński?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Brak danych pasujących do pytania.
[ŹRÓDŁA]
– brak


In [ ]:

tests = [
"ile jest dzieci?"
]
for q in tests:
    print("\nQ:", q)
    print(answer(q, k_ctx=10))



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Q: ile jest dzieci?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Brak danych pasujących do pytania.
[ŹRÓDŁA]
– brak


NameError: name 'ce_score' is not defined

In [ ]:
tests = [
"Jakie było przeciętne świadczenie emerytalne wypłacane przez KRUS w Polsce w pierwszym kwartale 2024 roku?",
"Jakie było przypisane i wpływy należności z tytułu składek na ubezpieczenie społeczne rolników w województwie podkarpackim, z uwzględnieniem składek finansowanych z budżetu państwa na ubezpieczenie emerytalno-rentowe za osoby sprawujące opiekę nad dzieckiem?",
"Jakie było przeciętne świadczenie w złotych zasiłków pogrzebowych po inwalidach wojennych wojskowych i osobach represjonowanych oraz członkach ich rodzin w czwartym kwartale 2024 roku?",
"Jaką kwotę wypłat w złotych stanowiły renty z tytułu niezdolności do pracy wypadkowe w transferach emerytalno-rentowych do Grecji?",
"Jaką liczbę płatników składek w Kasie Rolniczego Ubezpieczenia Społecznego (KRUS) odnotowano w Polsce 30 września 2024 roku?",
"Jaką liczbę ubezpieczonych zarejestrowano w Funduszu Emerytalno-Rentowym w pierwszym kwartale 2025 roku?",
"Jaką liczbę osób podlegających ubezpieczeniu zdrowotnemu w województwie lubuskim, które są domownikami pracującymi w gospodarstwach rolnych o powierzchni 6 hektarów lub większych, można przeliczyć na osoby pracujące w tych gospodarstwach?",
"Jaką przeciętną liczbę osób otrzymujących emerytury zarejestrowano w Portugalii?",
"Jaką przeciętną miesięczną liczbę świadczeniobiorców w I kwartale 2025 roku odnotowano w województwie pomorskim?",
"Jaką średnią miesięczną wartość emerytur i rent wypadkowych zanotowano w pierwszym kwartale 2024 roku?",
"Jakie były wydatki na świadczenia emerytalno-rentowe w województwie małopolskim?",
"Jaką przeciętną liczbę osób otrzymujących renty z tytułu niezdolności do pracy odnotowano w Hiszpanii?",
"Ilu rolników prowadzących wyłącznie działy specjalne produkcji rolnej podlega ubezpieczeniu zdrowotnemu w województwie lubuskim?",
"Jakie było przeciętne świadczenie zasiłku macierzyńskiego w województwie opolskim?",
"Jakie było przeciętne miesięczne świadczenie emerytalno-rentowe realizowane przez KRUS ogółem w województwie mazowieckim?",
"Jaką kwotę w złotych przeznaczono na świadczenia pieniężne dla cywilnych niewidomych ofiar działań wojennych w pierwszym kwartale 2025 roku?",
"Ile wniosków o przyznanie rodzicielskich świadczeń uzupełniających zostało rozpatrzonych w pierwszym kwartale 2025 roku w skali całego kraju?",
"Jaką liczbę wniosków o emerytury razem pozostało do załatwienia w następnym okresie?",
"Jaką kwotę w złotych stanowiły emerytury i renty realizowane przez KRUS z tytułu świadczeń zabiegowych dla robotników w czwartym kwartale 2024 roku?",
"Jakie były wydatki na emerytury i renty ogółem w pierwszym kwartale 2025 roku?",
"Jakie było przeciętne miesięczne świadczenie emerytalno-rentowe w województwie podlaskim?",
"Jakie było wartość świadczeń zleconych do wypłaty w Kasie Rolniczego Ubezpieczenia Społecznego w czwartym kwartale 2024 roku?",
"Jaką liczbę wniosków o przyznanie emerytury lub renty rodzinnej odnotowano w rekordzie dotyczącym rent rodzinnych?",
"Jakie było wskaźniki ściągalności składek na ubezpieczenie społeczne rolników w województwie śląskim?",
"Jaką liczbę współmałżonków zarejestrowano w ubezpieczeniu w trzecim kwartale 2020 roku?",
"Jaką liczbę wniosków o emerytury i renty zarejestrowano w województwie opolskim w pierwszym kwartale 2025 roku?",
"Jakie były wydatki na świadczenia emerytalno-rentowe w województwie kujawsko-pomorskim w Polsce?",
"Jaką przeciętną liczbę osób otrzymujących renty rodzinne odnotowano w USA?",
"Jakie było przeciętne miesięczne świadczenie emerytur rolniczych w czwartym kwartale 2024 roku?",
"Jaką średnią miesięczną liczbę emerytur i rent otrzymywali mieszkańcy województwa śląskiego w zakresie świadczeń rodzinnych, w tym wypadkowych, w nieokreślonym okresie?",
"Jaką kwotę wypłacono w zasiłkach pogrzebowych po członkach rodziny w całym kraju w czwartym kwartale 2024 roku?",
"Ile osób było ubezpieczonych z tytułu prowadzenia jednocześnie działalności rolniczej i pozarolniczej działalności gospodarczej w województwie wielkopolskim?",
"Jaką kwotę stanowiły składki finansowane z budżetu państwa na ubezpieczenie emerytalno-rentowe za osoby sprawujące opiekę nad dzieckiem w całym kraju?",
"Jaką liczbę wniosków o emerytury i renty pozostawiono do rozpatrzenia w następnym okresie w województwie podlaskim?",
"Ilu członków rodzin emerytów i rencistów podlega ubezpieczeniu zdrowotnemu w całej Polsce?",
"Ile osób ubezpieczonych w KRUS jest jednocześnie objętych ubezpieczeniem społecznym w ZUS z innego tytułu w województwie łódzkim?",
"Jakie było przeciętne miesięczne świadczenie emerytalne wypłacane w czwartym kwartale 2024 roku w Polsce, które dotyczyło emerytur za przekazane gospodarstwo rolne Państwu?",
"Jaką kwotę w złotych stanowiły renty wypadkowe w wypłatach emerytalno-rentowych przekazywanych do państw UE/EFTA i Wielkiej Brytanii razem?",
"Jakie było przeciętne miesięczne świadczenie uzupełniające dla osób niezdolnych do samodzielnej egzystencji w pierwszym kwartale 2025 roku w Kasie Rolniczego Ubezpieczenia Społecznego?"
"Jaką liczbę ubezpieczonych w okresie przyznania świadczenia pielęgnacyjnego lub specjalnego zasiłku opiekuńczego albo zasiłku dla opiekuna odnotowano w trzecim kwartale 2021 roku?"
"Jak wielu członków rodzin sprawujących opiekę nad dzieckiem było ubezpieczonych w drugim kwartale 2023 roku?",
"Ile osób ubezpieczonych w KRUS jest jednocześnie objętych ubezpieczeniem społecznym w ZUS z innego tytułu w województwie podlaskim?",
"Jaką liczbę ubezpieczonych zarejestrowano w Funduszu Składkowym w pierwszym kwartale 2024 roku?",
]
for q in tests:
    print("\nQ:", q)
    print(answer(q, k_ctx=6))


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Q: Jakie było przeciętne świadczenie emerytalne wypłacane przez KRUS w Polsce w pierwszym kwartale 2024 roku?


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Odpowiedź: W pierwszym kwartale 2024 roku przeciętne świadczenie emerytalne wypłacane przez KRUS w Polsce wynosiło 2184,49 złotych.
[ŹRÓDŁA]
- #[1] dataset=EMERYTURY I RENTY REALIZOWANE PRZEZ KRUS | source=all_data.csv | okres=2025-Q1 | region=ogółem | measure=przeciętne świadczenie w zł | typ=emerytury| value=2184.49 | ce=0.986 | rrf_sup=3 | rrf=0.0435
- #[2] dataset=EMERYTURY I RENTY REALIZOWANE PRZEZ KRUS | source=all_data.csv | okres=2025-Q1 | region=ogółem | measure=przeciętne świadczenie w zł | typ=renty rodzinne| value=2744.96 | ce=0.984 | rrf_sup=3 | rrf=0.0285
- #[3] dataset=EMERYTURY I RENTY REALIZOWANE PRZEZ KRUS | source=all_data.csv | okres=2025-Q1 | region=ogółem | measure=przeciętne świadczenie w zł | typ=ogółem| value=2178.42 | ce=0.978 | rrf_sup=3 | rrf=0.0325
- #[4] dataset=EMERYTURY I RENTY REALIZOWANE PRZEZ KRUS | source=all_data.csv | okres=2025-Q1 | region=ogółem | measure=przeciętne świadczenie w zł | typ=renty z tytułu niezdolności do pracy| value=2013.48 | ce=0

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Odpowiedź: W pierwszym kwartale 2025 roku w Kasie Rolniczego Ubezpieczenia Społecznego przeciętne miesięczne świadczenie uzupełniające dla osób niezdolnych do samodzielnej egzystencji wynosiło 168274.0 złotych. W trzecim kwartale 2021 roku w Kasie Rolniczego Ubezpieczenia Społecznego odnotowano 9572 ubezpieczonych w okresie przyznania świadczenia pielęgnacyjnego lub specjalnego zasiłku opiekuńczego albo zasiłku dla opiekuna. W drugim kwartale 2023 roku w Kasie Rolniczego Ubezpieczenia Społecznego odnotowano 0 członków rodzin sprawujących opiekę nad dzieckiem.
[ŹRÓDŁA]
- #[1] dataset=TABLICA 1 (25). ŚWIADCZENIA ZLECONE DO WYPŁATY KASIE ROLNICZEGO UBEZPIECZENIA SPOŁECZNEGO | source=all_data.csv | okres=2025-Q1 | region=ogółem | measure=2025-Q1 | typ=Przeciętna miesięczna liczba świadczeń ŚWIADCZENIE UZUPEŁNIAJĄCE DLA OSÓB NIEZDOLNYCH DO SAMODZIELNEJ EGZYSTENCJI| value=168274.0 | ce=0.944 | rrf_sup=2 | rrf=0.0328
- #[2] dataset=TABLICA 1 (25). ŚWIADCZENIA ZLECONE DO WYPŁATY KASIE ROLNICZE

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Odpowiedź: W województwie podlaskim w 2025 roku w KRUS było 1520 osób ubezpieczonych przy jednoczesnym objęciu ubezpieczeniem społecznym w ZUS z innego tytułu.
[ŹRÓDŁA]
- #[1] dataset=TABLICA 6 (35). LICZBA UBEZPIECZONYCH W KRUS PRZY JEDNOCZESNYM OBJĘCIU UBEZPIECZENIEM SPOŁECZNYM W ZUS Z INNEGO TYTUŁU 1)  WEDŁUG WOJEWÓDZTW | source=all_data.csv | okres=2025 | region=podlaskie | measure=ogółem | typ=ogółem| value=1520.0 | ce=0.980 | rrf_sup=3 | rrf=0.0397
- #[2] dataset=TABLICA 6 (35). LICZBA UBEZPIECZONYCH W KRUS PRZY JEDNOCZESNYM OBJĘCIU UBEZPIECZENIEM SPOŁECZNYM W ZUS Z INNEGO TYTUŁU 1)  WEDŁUG WOJEWÓDZTW | source=all_data.csv | okres=2025 | region=pomorskie | measure=ogółem | typ=ogółem| value=619.0 | ce=0.892 | rrf_sup=3 | rrf=0.0276
- #[3] dataset=TABLICA 6 (35). LICZBA UBEZPIECZONYCH W KRUS PRZY JEDNOCZESNYM OBJĘCIU UBEZPIECZENIEM SPOŁECZNYM W ZUS Z INNEGO TYTUŁU 1)  WEDŁUG WOJEWÓDZTW | source=all_data.csv | okres=2025 | region=śląskie | measure=ogółem | typ=ogółem| value=431.0 |

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Odpowiedź: W pierwszym kwartale 2024 roku w Funduszu Składkowym zarejestrowano 994 420 ubezpieczonych.
[ŹRÓDŁA]
- #[1] dataset=TABLICA 4 (29). WYPADKI I CHOROBY ZAWODOWE, Z TYTUŁU KTÓRYCH PRZYZNANO JEDNORAZOWE ODSZKODOWANIA WEDŁUG WOJEWÓDZTW | source=all_data.csv | okres=2024 | region=zachodniopomorskie | measure=Liczba wypadków na 1000 ubezpieczonych | typ=ogółem| value=1.5 | ce=0.025 | rrf_sup=1 | rrf=0.0123
- #[2] dataset=TABLICA 4 (29). WYPADKI I CHOROBY ZAWODOWE, Z TYTUŁU KTÓRYCH PRZYZNANO JEDNORAZOWE ODSZKODOWANIA WEDŁUG WOJEWÓDZTW | source=all_data.csv | okres=2024 | region=OGÓŁEM | measure=Liczba wypadków na 1000 ubezpieczonych | typ=ogółem| value=2.0 | ce=0.024 | rrf_sup=1 | rrf=0.0104
- #[3] dataset=TABLICA 4 (29). WYPADKI I CHOROBY ZAWODOWE, Z TYTUŁU KTÓRYCH PRZYZNANO JEDNORAZOWE ODSZKODOWANIA WEDŁUG WOJEWÓDZTW | source=all_data.csv | okres=2024 | region=kujawsko-pomorskie | measure=Liczba wypadków na 1000 ubezpieczonych | typ=ogółem| value=3.0 | ce=0.023 | rrf_sup=1 | rrf=0